In [ ]:
file_path = '../dataset/THUCNews/彩票/256822.txt'
with open(file_path, "r", encoding='utf-8') as f:
    str = f.read()

In [ ]:
new_str = str.replace(' ', '').replace('\n\u3000\u3000', '')
print(new_str)
print(len(new_str))

In [ ]:
from GPT2 import GPT2Model, GPT2Tokenizer
tokenizer = GPT2Tokenizer(
    'GPT2/bpe/vocab.json',
    'GPT2/bpe/chinese_vocab.model',
    max_len=512)
tokenized_str = tokenizer.encode(new_str)
print(tokenized_str)

In [ ]:
str_eng = "This is a good example of training GPT2 to solve this problem"
tokenized_str = tokenizer.encode(str_eng)
print(tokenized_str)

In [ ]:
import os
def get_folders(path):
    folders = []
    g = os.walk(path)  
    for path,dir_list,file_list in g:  
        for dir_name in dir_list:
            folders.append(dir_name)
    return folders
def get_files(path):
    files = []
    g = os.walk(path)  
    for path,dir_list,file_list in g:  
        for file_name in file_list:
            files.append(file_name)
    return files

In [ ]:
'''
    每个类别放到一个txt里
'''

from tqdm import tqdm

path = '../dataset/THUCNews'
save_path = '../dataset/THUCNews_processed'
folders = get_folders(path)
print(folders)
for folder in folders:
    save_list = []
    files = get_files(os.path.join(path, folder))
    for file in tqdm(files):
        with open(os.path.join(path, folder, file),"r", encoding='utf-8') as f:
            str = f.read()
            new_str = str.replace(' ', '').replace('\n\u3000\u3000', '')
            save_list.append(new_str)
    with open(os.path.join(save_path, folder + '.txt'),"w+", encoding='utf-8') as f:
        for line in save_list:
            f.write(line + '\n')

In [ ]:
'''
    二分类json数据集
'''

from tqdm import tqdm
import random
import json

path = '../dataset/THUCNews'
save_path = '../dataset/THUCNews_processed'
folders = get_folders(path)
print(folders)

objs_financial = []
objs_others = []

for folder in folders:
    if folder in ['财经', '彩票', '股票']:
        files = get_files(os.path.join(path, folder))
        for file in tqdm(files):
            with open(os.path.join(path, folder, file),"r", encoding='utf-8') as f:
                obj = dict()
                str = f.read()
                new_str = str.replace(' ', '').replace('\n\u3000\u3000', '')
                obj['label_desc'] = 'news_finance'
                obj['sentence'] = new_str
                objs_financial.append(obj)
    else:
        label = 'news_' + folder
        files = get_files(os.path.join(path, folder))
        for file in tqdm(files):
            with open(os.path.join(path, folder, file),"r", encoding='utf-8') as f:
                obj = dict()
                str = f.read()
                new_str = str.replace(' ', '').replace('\n\u3000\u3000', '')
                obj['label_desc'] = label
                obj['sentence'] = new_str
                objs_others.append(obj)

random.shuffle(objs_financial)
random.shuffle(objs_others)

num = min(len(objs_financial), len(objs_others))
objs = []
objs.extend(objs_financial[:num])
objs.extend(objs_others[:num])

random.shuffle(objs)

num = int(len(objs)*0.9)
objs_train = objs[:num]
objs_test = objs[num:]

filename = '../dataset/THUCNews_processed/train_financial.json'
with open(filename, 'w', encoding='utf-8') as fout:
    for obj in objs_train:
        line = json.dumps(obj, ensure_ascii=False)
        fout.write(line)
        fout.write('\n')

filename = '../dataset/THUCNews_processed/test_financial.json'
with open(filename, 'w', encoding='utf-8') as fout:
    for obj in objs_test:
        line = json.dumps(obj, ensure_ascii=False)
        fout.write(line)
        fout.write('\n')